In [3]:
import numpy as np
import pickle
import pandas as pd 
import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

read file

In [4]:
file = open('covid_dataset.pkl', 'rb')
checkpoint = pickle.load(file)
file.close()

make data sets.  Fill in missing value with imputer (multivariate model).  Normalize data

In [206]:
def norm(X):
    scaler = MinMaxScaler()
    scaler.fit(X)
    return scaler.transform(X)

def apply_norm_fix(X, should_norm):
    X = np.array(X, dtype=np.float64)
    imp = IterativeImputer(max_iter=1000, random_state=0)
    imp.fit(X)
    X = imp.transform(X)
    if should_norm:
        X = norm(X)
    return X

X_train  = apply_norm_fix(checkpoint["X_train"], False)
y_train = np.array(checkpoint["y_train_log_pos_cases"], dtype=np.float64)

X_val = apply_norm_fix(checkpoint["X_val"], False)
y_val = np.array(checkpoint["y_val_log_pos_cases"], dtype=np.float64)

big_X = apply_norm_fix(np.array(list(X_train) + list(X_val)), False)
big_y = np.array(list(y_train) + list(y_val), dtype=np.float64)

X_test = apply_norm_fix(checkpoint["X_test"], False)

Use algorithm (Gradient Boosted Tree with squared loss).  Hyper parameter search using randomized search.

In [210]:
params = {'learning_rate': np.linspace(0.1, 0.7, 1000, dtype=np.float64), 'n_estimators': np.linspace(10, 120, 110, dtype=np.int32), 'max_depth': [1], 'random_state' : [0], 'loss': ['lad']}
#n_est 110, lr 0.128
algorithm = GBR(random_state=0, n_estimators=10000, loss='lad', learning_rate=0.45, max_depth=3)
x = X_train
y = y_train
model = algorithm.fit(x, y)
preds = model.predict(X_val)
mse = mean_squared_error(y_val, preds)
print(mse)

test_pred = model.predict(X_test)
pd.DataFrame(test_pred).to_csv("predictions.csv", header=["cases"], index_label="id")

0.8694415572117508


In [163]:
print(model.best_params_)

{'random_state': 0, 'n_estimators': 28, 'max_depth': 1, 'loss': 'lad', 'learning_rate': 0.5534534534534534}
